<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Машинное обучение и майнинг данных"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# Занятие 8. Apache Spark
## Часть 1. Введение

In [2]:
%pylab inline
import numpy as np
import re

Populating the interactive namespace from numpy and matplotlib


#### Немного функционального программирования

In [3]:
xs = range(5)

In [4]:
map(lambda x: x*x, xs)

[0, 1, 4, 9, 16]

In [5]:
reduce(lambda acc, x: acc  / (x + 1), xs, 1.0)

0.008333333333333333

In [6]:
filter(lambda x: x % 3 == 1, xs)

[1, 4]

## Базовые преобразования RDD

In [7]:
from pyspark import SparkContext, SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)

### Создание RDD

In [2]:
nums = sc.parallelize(xrange(0, 25))
text = sc.textFile("../../data/Sherlock.txt")

NameError: name 'sc' is not defined

In [9]:
text.take(5)

[u'',
 u'',
 u'',
 u'',
 u'                        THE ADVENTURES OF SHERLOCK HOLMES']

### Простые операции с RDD

In [10]:
nums.map(lambda x: x + 1).collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [11]:
words = text.flatMap(lambda x: re.findall("[a-zA-z']+", x)).map(lambda x: x.strip().lower()).persist()
words.take(10)

[u'the',
 u'adventures',
 u'of',
 u'sherlock',
 u'holmes',
 u'arthur',
 u'conan',
 u'doyle',
 u'table',
 u'of']

In [12]:
nums.filter(lambda x: x % 2 == 0).collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]

In [13]:
nums.flatMap(lambda x: [x] if x % 2 == 0 else []).collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]

In [14]:
nums.aggregate("", lambda acc, x: acc + str(x), lambda acc1, acc2: acc1 + " " + acc2)

' 01234567891011 12131415161718192021222324'

In [15]:
nums.repartition(1).aggregate("", lambda acc, x: acc + str(x), lambda acc1, acc2: acc1 + " " + acc2)

' 0123456789101112131415161718192021222324'

In [16]:
nums.fold(0, lambda a, b: a + b)

300

In [17]:
nums.reduce(lambda a, b: a + b)

300

In [18]:
nums.max(), nums.min()

(24, 0)

### Операции парных  RDD

In [19]:
words.map(lambda w: (w, 1)).groupByKey().map(lambda (w, c): (w, sum(list(c)))).take(10)

[(u'aided', 1),
 (u'hollowed', 1),
 (u'augustine', 1),
 (u'believed', 3),
 (u'hats', 1),
 (u'crumpled', 3),
 (u'yellow', 15),
 (u'four', 38),
 (u'grosvenor', 2),
 (u'sleep', 11)]

In [20]:
count = words.map(lambda w: (w, 1)).reduceByKey(lambda c1, c2: c1 + c2).persist()
count.take(20)

[(u'aided', 1),
 (u'hollowed', 1),
 (u'augustine', 1),
 (u'believed', 3),
 (u'hats', 1),
 (u'crumpled', 3),
 (u'yellow', 15),
 (u'four', 38),
 (u'grosvenor', 2),
 (u'sleep', 11),
 (u'keeper', 11),
 (u"friend's", 10),
 (u'scheming', 1),
 (u'pheasant', 1),
 (u'saved', 9),
 (u'swag', 1),
 (u'pointing', 7),
 (u'granting', 1),
 (u'paris', 1),
 (u'characterises', 1)]

In [21]:
wordsLen = words.distinct().map(lambda w: (w, len(w))).persist()
wordsLen.take(10)

[(u'aided', 5),
 (u'hollowed', 8),
 (u'augustine', 9),
 (u'believed', 8),
 (u'hats', 4),
 (u'crumpled', 8),
 (u'yellow', 6),
 (u'four', 4),
 (u'grosvenor', 9),
 (u'sleep', 5)]

In [22]:
count.cogroup(wordsLen).map(lambda (w, (counts, lens)): (w, list(counts), list(lens)) ).take(10)

[(u'aided', [1], [5]),
 (u'four', [38], [4]),
 (u'sleep', [11], [5]),
 (u'mansion', [1], [7]),
 (u'pheasant', [1], [8]),
 (u'saved', [9], [5]),
 (u'kempt', [1], [5]),
 (u'paris', [1], [5]),
 (u'characterises', [1], [13]),
 (u'originality', [1], [11])]

In [1]:
count.join(wordsLen).take(10)

NameError: name 'count' is not defined